# Atlas

In this notebook, you will learn how to use `atlas` to conduct autonomous experimentation.

Notes:
- when executing the first cell, you will receive a warning ("This notebook was not authored by Google"). Please select "Run Anyway" to be able to run the cells of the notebook.

In [1]:
# we start by installing a couple of dependencies
print('...')

...


In [2]:
# finally, we install atlas itself
print('...')

...


## Proof-of-concept optimization

## Optimization of mixed-parameter spaces


## Optimization with a-priori known constraints

## Optimization with a-priori unknown constraints

## Multi-objective optimization

## Robust optimization with Golem

## Optimization for a generalizable set of parameters